<a href="https://colab.research.google.com/github/anik199/House_Prices_Advanced_Regression/blob/main/House_Prices_ART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step:1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('/content/train.csv', index_col='Id')
X_test_full = pd.read_csv('/content/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 20 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

## Step:2

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Step:3


##Model:0

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostRegressor
# Define the model
my_model_0 = CatBoostRegressor()

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_0)
                             ])
                                    

# Fit the model
my_model_0=my_pipeline.fit(X_train, y_train)

# Get predictions
predictions_0 = my_pipeline.predict(X_valid)

# Calculate MAE
mse_0 = mean_squared_error(predictions_0, y_valid)
print("RMSE:" , mse_0)

# Run prediction on the Kaggle test set.
preds_test_0 = my_pipeline.predict(X_test)

### Model:1

In [ ]:
# I found this best alpha through cross-validation.
best_alpha = 350
# Define the model
my_model_1= Lasso(alpha=best_alpha, max_iter=50000)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_1)
                             ])

# Fit the model
my_model_1=my_pipeline.fit(X_train, y_train)

# Get predictions
predictions_1 = my_pipeline.predict(X_valid)

# Calculate MAE
mse_1 = mean_squared_error(predictions_1, y_valid)
print("RMSE:" , mse_1)

# Run prediction on the Kaggle test set.
preds_test_1 = my_pipeline.predict(X_test)

### Model:2

In [ ]:
# Define the model
my_model_2 = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0, gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_2)
                             ])
                                    

# Fit the model
my_model_2=my_pipeline.fit(X_train, y_train)

# Get predictions
predictions_2 = my_pipeline.predict(X_valid)

# Calculate MAE
mse_2 = mean_squared_error(predictions_2, y_valid)
print("RMSE:" , mse_2)

# Run prediction on the Kaggle test set.
preds_test_2 = my_pipeline.predict(X_test)

### Model:3

In [ ]:
# Define the model
my_model_3 = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_3)
                             ])

# Fit the model
my_model_3=my_pipeline.fit(X_train, y_train)

# Get predictions
predictions_3 = my_pipeline.predict(X_valid)

# Calculate MAE
mse_3 = mean_squared_error(predictions_3, y_valid)
print("RMSE:" , mse_3)

# Run prediction on the Kaggle test set.
preds_test_3 = my_pipeline.predict(X_test)

### Model:4

In [ ]:
# Define the model
my_model_4 = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.05, 
                                       max_depth=4, max_features='sqrt', 
                                       min_samples_leaf=15, min_samples_split=10, 
                                       loss='huber', random_state =0) 


# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model_4)
                             ])
                                


    
# Fit the model
my_model_4=my_pipeline.fit(X_train, y_train)

# Get predictions
predictions_4 = my_pipeline.predict(X_valid)

# Calculate MAE
mse_4 = mean_squared_error(predictions_4, y_valid)
print("RMSE:" , mse_4)

# Run prediction on the Kaggle test set.
preds_test_4= my_pipeline.predict(X_test)

### Predictions

In [ ]:
# Combined Best Models Average
preds_test= (preds_test_0 + preds_test_2 + preds_test_3) / 3
preds_test= my_pipeline.predict(X_test)

### Make Prediction Dataframe

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)